# 📊 Text-to-SQL: Generación de Consultas SQL desde Lenguaje Natural

Objetivo: implementar sistemas NL2SQL (Natural Language to SQL) robustos usando LLMs, con validación, seguridad y optimización.

- Duración: 90-120 min
- Dificultad: Media/Alta
- Prerrequisitos: GenAI 01, SQL intermedio, schemas de base de datos

## 1. Caso base: prompt simple

In [ ]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def nl_to_sql_simple(question: str) -> str:
    prompt = f'''
Convierte esta pregunta en SQL:

Pregunta: {question}

SQL:
'''
    resp = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    return resp.choices[0].message.content

print(nl_to_sql_simple('Muestra las 10 ventas más recientes'))

## 2. Mejorado: con contexto de schema

In [ ]:
schema_context = '''
Tablas disponibles:

ventas:
  - venta_id (int, PK)
  - fecha (date)
  - cliente_id (int, FK)
  - producto_id (int, FK)
  - cantidad (int)
  - total (decimal)

clientes:
  - cliente_id (int, PK)
  - nombre (varchar)
  - email (varchar)
  - pais (varchar)

productos:
  - producto_id (int, PK)
  - nombre (varchar)
  - categoria (varchar)
  - precio (decimal)
'''

def nl_to_sql(question: str, schema: str = schema_context, dialect='postgresql') -> str:
    prompt = f'''
Eres un experto en SQL. Genera una consulta {dialect} válida para responder la pregunta.

Schema de base de datos:
{schema}

Pregunta: {question}

Genera SOLO la consulta SQL, sin explicaciones adicionales.
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    return resp.choices[0].message.content.strip().replace('```sql','').replace('```','')

query = nl_to_sql('¿Cuál es el top 5 de productos más vendidos por categoría?')
print(query)

## 3. Validación y seguridad

In [ ]:
import re
import sqlparse

def is_safe_query(sql: str) -> tuple[bool, str]:
    """Valida que la consulta sea de solo lectura y segura."""
    sql_lower = sql.lower()
    # Bloquear operaciones de escritura
    dangerous = ['insert', 'update', 'delete', 'drop', 'create', 'alter', 'truncate', 'exec', 'execute']
    for kw in dangerous:
        if re.search(rf'\b{kw}\b', sql_lower):
            return False, f'Operación bloqueada: {kw}'
    
    # Solo permitir SELECT
    if not sql_lower.strip().startswith('select'):
        return False, 'Solo se permiten consultas SELECT'
    
    return True, 'OK'

test_queries = [
    'SELECT * FROM ventas LIMIT 10',
    'DELETE FROM ventas WHERE fecha < \'2020-01-01\'',
    'SELECT COUNT(*) FROM clientes'
]

for q in test_queries:
    safe, msg = is_safe_query(q)
    print(f'{"✅" if safe else "❌"} {msg}: {q[:50]}')

## 4. Ejecución segura con SQLite demo

In [ ]:
import sqlite3
import pandas as pd

# Setup demo DB
conn = sqlite3.connect(':memory:')
conn.executescript('''
CREATE TABLE ventas (venta_id INT, fecha TEXT, cliente_id INT, producto_id INT, cantidad INT, total REAL);
INSERT INTO ventas VALUES (1,'2025-10-01',10,101,2,200.0),(2,'2025-10-02',11,102,1,50.0),(3,'2025-10-03',10,101,1,100.0);
CREATE TABLE productos (producto_id INT, nombre TEXT, categoria TEXT);
INSERT INTO productos VALUES (101,'Laptop','Electronics'),(102,'Mouse','Electronics');
''')

def execute_nl_query(question: str):
    sql = nl_to_sql(question)
    print(f'SQL generado: {sql}\n')
    safe, msg = is_safe_query(sql)
    if not safe:
        return f'❌ Consulta bloqueada: {msg}'
    try:
        df = pd.read_sql_query(sql, conn)
        return df
    except Exception as e:
        return f'Error ejecutando SQL: {e}'

result = execute_nl_query('Muestra el total de ventas por producto')
print(result)

## 5. Few-shot con ejemplos

In [ ]:
few_shot_examples = '''
Ejemplos:

Q: ¿Cuántas ventas hubo ayer?
SQL: SELECT COUNT(*) FROM ventas WHERE fecha = CURRENT_DATE - INTERVAL '1 day';

Q: Top 3 clientes por gasto total
SQL: SELECT c.nombre, SUM(v.total) as gasto FROM ventas v JOIN clientes c ON v.cliente_id=c.cliente_id GROUP BY c.nombre ORDER BY gasto DESC LIMIT 3;

Q: Productos sin ventas en octubre
SQL: SELECT p.nombre FROM productos p LEFT JOIN ventas v ON p.producto_id=v.producto_id AND v.fecha >= '2025-10-01' AND v.fecha < '2025-11-01' WHERE v.venta_id IS NULL;
'''

def nl_to_sql_fewshot(question: str) -> str:
    prompt = f'''
Genera SQL para responder preguntas sobre ventas.

{schema_context}

{few_shot_examples}

Ahora genera SQL para:
Q: {question}
SQL:
'''
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    return resp.choices[0].message.content.strip().replace('```sql','').replace('```','')

query = nl_to_sql_fewshot('Ventas por categoría en los últimos 7 días')
print(query)

## 6. Optimización y caché

In [ ]:
from functools import lru_cache
import hashlib

@lru_cache(maxsize=128)
def cached_nl_to_sql(question: str) -> str:
    return nl_to_sql(question)

# Llamadas repetidas usan caché
q = '¿Cuál es el total de ventas del mes actual?'
print('Primera llamada:')
print(cached_nl_to_sql(q))
print('\nSegunda llamada (desde caché):')
print(cached_nl_to_sql(q))

## 7. Buenas prácticas

- **Schema completo**: incluye tipos, PKs, FKs, índices.
- **Ejemplos representativos**: few-shot con casos edge.
- **Validación estricta**: whitelist de operaciones permitidas.
- **Timeout y límites**: evita consultas costosas.
- **Logging**: registra pregunta, SQL generado, resultado.
- **Feedback loop**: almacena correcciones humanas para fine-tuning.

## 8. Ejercicios

1. Agrega soporte multi-idioma (inglés/español).
2. Implementa un sistema de aprobación humana para SQL complejos.
3. Crea un dashboard Streamlit donde usuarios escriban preguntas y vean resultados.
4. Añade explicación del SQL generado en lenguaje natural.